# Training the LSTM on your Data

This notebook will take you through the steps necessary to train an LSTM to recognize ICD-9 codes, or items from similar dictionaries, from free text.

## Setup

### Imports

Make sure that the below packages are installed on the server on which this program will run.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
import tensorflow as tf
from simpleLSTMWithNNetModel import Model, reloadModel
from trainModel import trainModel
import os
import pprint
import numpy as np
import pickle
pp = pprint.PrettyPrinter(indent=4)

### Directories

Executing the following cell will prompt the user to type in the directories corresponding to the training and validation sets, the vocabulary, and the word vector mappings. Defaults are given in the comments below.

Note that the training and test data needs to have one column dedicated to free text (`noteIdx`) and another dedicated to top-level ICD-9 codes (`codeIdx`) associated with each patient. Preferably, the latter should be strung together using '-' as the delimiter (e.g. for patient 1, 1-2-6-4).

Please make sure that the parameters such as the embed size, maximum note length, learning rate, number of maximum training epochs, batch size, hidden layer size, number of neural net layers, and probabilities are to your specification.

In [ ]:
data_train = raw_input('What is the path to the training data? ') #default: data/icd9NotesDataTable_train.csv
data_valid = raw_input('What is the path to the validation data? ') #default: data/icd9NotesDataTable_valid.csv
vocab = raw_input('What is the path to the vocabulary? ') #default: data/icd9Vocab.txt
wordVecs = raw_input('What is the path to the vocabulary? ') #data/newgloveicd9.txt. These are length 300 word vectors from GloVE

NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 300 # this should correspond to the length of the word vectors
maxAllowedNoteLength = 1000
max_grad_norm = 5
codeIdx = raw_input('Which column contains top-level ICD-9 codes (outputs) in the training and test data? ') #default: 9
textIdx = raw_input('Which column contains notes/text (inputs) in the training and test data? ') #default: 6
learning_rate = 0.001
training_epochs = 100
batch_size = 256
n_hidden = 200
output_keep_prob = 0.5
input_keep_prob = 1
numLayers = 1

Here, make sure that the output path is specified as you would like. By default, the program saves the output in a folder with the name of your choice within the folder `results`.

If there exists a folder with results that you would like to load again, use that here.

In [ ]:
output_path = raw_input('Where are models and performances (to be) saved? ')
output_path = os.path.join('results', output_path)
if output_path == 'results/':
    output_path = 'results/temp'
if not os.path.exists(output_path):
    os.makedirs(output_path)

## Initialization

Executing the following cell will ask whether or not there is a previously saved model; if not, the model will train features from scratch, and if so, the features will be loaded.

In [ ]:
sizeList = [n_hidden, 150, 75] # these are the weights we wil be using

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    """
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")
            
prev_model = query_yes_no("Is there a previously trained model?")

if prev_model:
    helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx,
    helperLoadPath = output_path)
else:
    helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx)
    
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(output_path) # token2id and max length saved to output_path
sizeList.append(helper.n_labels)

total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch: %d'%(total_batches))
print('Maximum note length: %d'%(helper.max_length))
print('Number of ICD-9 codes: %d'%(helper.n_labels))
print('There are a total of: {} ICD-9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec length = %d'%(embeddings.shape))

The following cell initializes the dictionary of hyperparameters for the model that fully describe the model.

In [ ]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers,
                 'sizeList':sizeList}
pp.pprint(hyperParamDict)
with open(os.path.join(output_path, 'hyperParamDict.pickle'), 'wb') as handle:
    pickle.dump(hyperParamDict, handle, protocol = 2)
    #dumping with 2 because ALTUD uses python 2.7 right now.

## Training

Finally, the model is trained (be wary - it will take some time; on an Amazon Deep Learning AMI, it took around an hour to train)...

In [ ]:
from trainModel import trainModel
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trainModel(helperObj = helper, embeddings = embeddings, hyperParamDict = hyperParamDict, 
          xDev = xDev, xTrain = xTrain, yDev = yDev, yTrain = yTrain, 
           lastTrueWordIdx_dev = lastTrueWordIdx_dev, 
           lastTrueWordIdx_train = lastTrueWordIdx_train,
           training_epochs = training_epochs, 
           output_path = output_path, batchSizeTrain = batch_size,
           sizeList = sizeList,
           maxIncreasingLossCount = 100, batchSizeDev = 1500, chatty = True)

and the session closed. You should be able to see your results in the `output_path` directory you specified earlier.

To evaluate the results and generate plots and such, please check out `predictionEvaluation.ipynb` in the same repository.

In [ ]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()